# GWAS Manhattan Plot Exercise

This exercise will guide you through creating Manhattan plots from GWAS summary statistics and interpreting the results.

## Learning Objectives
- Load and explore GWAS summary statistics
- Create interactive Manhattan plots
- Interpret genomic regions of interest
- Understand significance thresholds

## Exercise Setup

First, let's install the required packages and load our data:

In [ ]:
# Install required packages
# !pip install pandas numpy matplotlib seaborn plotly bokeh

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from scipy import stats

# Set plotting style
plt.style.use('seaborn')
sns.set_palette("husl")

# Enable inline plotting for Jupyter
%matplotlib inline

print("Setup complete!")

## Part 1: Load and Explore GWAS Data

Let's load the GWAS summary statistics and explore the data structure:

In [ ]:
# Load GWAS summary statistics
gwas_df = pd.read_csv('../Data/sample_gwas_summary_stats.csv')

# Display first few rows
print("First 10 rows of GWAS data:")
display(gwas_df.head(10))

# Basic statistics
print("\nData overview:")
print(f"Total SNPs: {len(gwas_df):,}")
print(f"Chromosomes represented: {sorted(gwas_df['CHR'].unique())}")
print(f"P-value range: {gwas_df['P'].min():.2e} - {gwas_df['P'].max():.2e}")
print(f"SNPs with p < 5e-8: {len(gwas_df[gwas_df['P'] <= -np.log10(5e-8)])}")
print(f"Most significant SNP: {gwas_df.loc[gwas_df['P'].idxmax(), 'SNP']} (p = {10**-gwas_df['P'].max():.2e})")

## Question 1:
What is the most significant SNP in this dataset? What chromosome is it on? What is its p-value?

In [ ]:
# Your answer here
# most_sig_snp = gwas_df.loc[gwas_df['P'].idxmax()]
# print(f"Most significant SNP: {most_sig_snp['SNP']}")
# print(f"Chromosome: {most_sig_snp['CHR']}")
# print(f"Position: {most_sig_snp['BP']:,}")
# print(f"P-value: {10**-most_sig_snp['P']:.2e}")
# print(f"Odds ratio: {most_sig_snp['OR']:.2f}")

## Part 2: Create a Basic Manhattan Plot

Let's create a Manhattan plot using matplotlib:

In [ ]:
# Function to create Manhattan plot
def create_manhattan_plot(df, title="GWAS Manhattan Plot"):
    """
    Create a Manhattan plot from GWAS summary statistics
    
    Parameters:
    df: DataFrame with columns CHR, BP, P, SNP
    title: Plot title
    """
    # Prepare data
    gwas_data = df.copy()
    
    # Calculate cumulative position for x-axis
    gwas_data['cum_pos'] = 0
    chrom_starts = {}
    
    for chr_num in sorted(gwas_data['CHR'].unique()):
        chrom_data = gwas_data[gwas_data['CHR'] == chr_num]
        start_pos = gwas_data[gwas_data['CHR'] < chr_num]['BP'].max() if chr_num > 1 else 0
        chrom_starts[chr_num] = start_pos
        gwas_data.loc[gwas_data['CHR'] == chr_num, 'cum_pos'] = chrom_data['BP'] + start_pos
    
    # Create plot
    fig, ax = plt.subplots(figsize=(15, 8))
    
    # Color scheme for chromosomes
    colors = ['#1f77b4', '#ff7f0e'] * 11  # Alternate blue and orange
    chr_colors = {chr_num: colors[chr_num-1] for chr_num in gwas_data['CHR'].unique()}
    
    # Plot each chromosome
    for chr_num in sorted(gwas_data['CHR'].unique()):
        chr_data = gwas_data[gwas_data['CHR'] == chr_num]
        ax.scatter(chr_data['cum_pos'], chr_data['P'], 
                  c=chr_colors[chr_num], s=3, alpha=0.8,
                  label=f'Chr {chr_num}')
    
    # Add significance thresholds
    ax.axhline(y=-np.log10(5e-8), color='red', linestyle='--', linewidth=1, 
               label=f'Genome-wide significance (5×10⁻⁸)')
    ax.axhline(y=-np.log10(1e-5), color='orange', linestyle='--', linewidth=1,
               label=f'Suggestive significance (1×10⁻⁵)')
    
    # Chromosome labels at centers
    chrom_centers = []
    chrom_labels = []
    for chr_num in sorted(gwas_data['CHR'].unique()):
        chr_data = gwas_data[gwas_data['CHR'] == chr_num]
        center_pos = chr_data['cum_pos'].mean()
        chrom_centers.append(center_pos)
        chrom_labels.append(f'{chr_num}')
    
    ax.set_xticks(chrom_centers)
    ax.set_xticklabels(chrom_labels)
    
    # Labels and title
    ax.set_xlabel('Chromosome', fontsize=12)
    ax.set_ylabel('-log₁₀(p-value)', fontsize=12)
    ax.set_title(title, fontsize=14, fontweight='bold')
    
    # Legend (only show chromosome legend for first few)
    ax.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
    
    plt.tight_layout()
    return fig, ax

# Create and display Manhattan plot
fig, ax = create_manhattan_plot(gwas_df, "GWAS Manhattan Plot - Sample Dataset")
plt.show()

## Part 3: Interactive Manhattan Plot

Now let's create an interactive version using Plotly:

In [ ]:
# Create interactive Manhattan plot
def create_interactive_manhattan(df, title="Interactive GWAS Manhattan Plot"):
    # Prepare data - add cumulative position
    plot_data = df.copy()
    
    # Calculate cumulative positions for continuous x-axis
    plot_data['cum_pos'] = 0
    chrom_starts = {}
    current_pos = 0
    
    for chr_num in sorted(plot_data['CHR'].unique()):
        chrom_data = plot_data[plot_data['CHR'] == chr_num]
        chrom_starts[chr_num] = current_pos
        plot_data.loc[plot_data['CHR'] == chr_num, 'cum_pos'] = chrom_data['BP'] + current_pos
        current_pos = plot_data[plot_data['CHR'] == chr_num]['cum_pos'].max() + 1e7  # Add gap between chromosomes
    
    # Color scheme
    colors = px.colors.qualitative.Set1
    chr_colors = {chr_num: colors[(chr_num-1) % len(colors)] for chr_num in plot_data['CHR'].unique()}
    
    # Create plot
    fig = go.Figure()
    
    # Add traces for each chromosome
    for chr_num in sorted(plot_data['CHR'].unique()):
        chr_data = plot_data[plot_data['CHR'] == chr_num]
        fig.add_trace(go.Scatter(
            x=chr_data['cum_pos'],
            y=chr_data['P'],
            mode='markers',
            name=f'Chr {chr_num}',
            marker=dict(
                color=chr_colors[chr_num],
                size=4,
                opacity=0.7
            ),
            hovertemplate=
                '<b>Chr %{customdata}</b><br>' +
                'SNP: %{text}<br>' +
                'Position: %{x:,}<br>' +
                '-log₁₀(p): %{y:.2f}<br>' +
                'OR: %{customdata[1]:.2f}<br>' +
                '<extra></extra>',
            customdata=chr_data[['CHR', 'OR']],
            text=chr_data['SNP']
        ))
    
    # Add significance lines
    fig.add_hline(y=-np.log10(5e-8), line_dash="solid", line_color="red",
                  annotation_text="Genome-wide (5e-8)")
    fig.add_hline(y=-np.log10(1e-5), line_dash="dash", line_color="orange", 
                  annotation_text="Suggestive (1e-5)")
    
    # Update layout
    fig.update_layout(
        title=title,
        xaxis_title='Genomic Position',
        yaxis_title='-log₁₀(p-value)',
        height=600,
        showlegend=True,
        # Hide x-axis tick labels (too crowded)
        xaxis=dict(showticklabels=False)
    )
    
    return fig

# Create interactive plot
interactive_fig = create_interactive_manhattan(gwas_df)
interactive_fig.show()

## Question 2:
How many genome-wide significant associations (p < 5×10⁻⁸) are there in this dataset? Which chromosomes do they occur on?

In [ ]:
# Your analysis here
# genome_wide_snps = gwas_df[gwas_df['P'] > -np.log10(5e-8)]
# print(f"Number of genome-wide significant SNPs: {len(genome_wide_snps)}")
# print("\nBy chromosome:")
# print(genome_wide_snps.groupby('CHR').size())

## Part 4: QQ Plot Analysis

Let's create a QQ plot to assess whether our results show inflation:

In [ ]:
# Create QQ plot
def create_qq_plot(df, title="GWAS QQ Plot"):
    fig, ax = plt.subplots(figsize=(8, 8))
    
    # Observed p-values (sorted)
    observed = np.sort(df['P'])[::-1]  # Sort descending for plotting
    
    # Expected p-values under null
    n_snps = len(df)
    expected = -np.log10(np.arange(1, n_snps + 1) / (n_snps + 1))
    
    # Plot
    ax.scatter(expected, observed, alpha=0.5, s=2, color='blue')
    
    # Diagonal line (expected = observed)
    max_val = max(max(observed), max(expected))
    ax.plot([0, max_val], [0, max_val], 'r--', linewidth=1, label='Expected')
    
    # Calculate lambda (inflation factor)
    lambda_gc = np.median(10 ** -observed) / np.median(10 ** -expected)
    
    # Labels and title
    ax.set_xlabel('Expected -log₁₀(p-value)')
    ax.set_ylabel('Observed -log₁₀(p-value)')
    ax.set_title(f'{title}\nλ = {lambda_gc:.3f}')
    ax.legend()
    ax.grid(True, alpha=0.3)
    
    # Equal aspect ratio
    ax.set_aspect('equal')
    
    return fig, ax, lambda_gc

# Create QQ plot
qq_fig, qq_ax, lambda_val = create_qq_plot(gwas_df, "QQ Plot for Sample GWAS Data")
plt.show()

print(f"Genomic inflation factor λ: {lambda_val:.3f}")

if lambda_val > 1.05:
    print("Warning: Evidence of systematic bias or polygenicity")
elif lambda_val < 0.95:
    print("Evidence of deflation (possibly due to filtering)")
else:
    print("No significant inflation or deflation detected")

## Part 5: Regional Association Plot

Let's create a regional plot around the most significant SNP:

In [ ]:
# Regional association plot
def create_regional_plot(df, snp_id, window_size=500000, title="Regional Association Plot"):
    """
    Create a regional association plot around a specific SNP
    
    Parameters:
    df: GWAS DataFrame
    snp_id: SNP identifier to center on
    window_size: Size of window around SNP (in bp)
    title: Plot title
    """
    # Find the SNP
    snp_data = df[df['SNP'] == snp_id]
    if len(snp_data) == 0:
        print(f"SNP {snp_id} not found")
        return None
    
    snp_info = snp_data.iloc[0]
    chr_num = snp_info['CHR']
    snp_pos = snp_info['BP']
    
    # Get regional data
    regional_df = df[
        (df['CHR'] == chr_num) & 
        (df['BP'] >= snp_pos - window_size) & 
        (df['BP'] <= snp_pos + window_size)
    ]
    
    if len(regional_df) == 0:
        print(f"No SNPs found in region around {snp_id}")
        return None
    
    # Create positions relative to index SNP
    regional_df = regional_df.copy()
    regional_df['rel_pos'] = regional_df['BP'] - snp_pos
    
    # Create plot
    fig, ax1 = plt.subplots(figsize=(12, 8))
    
    # Scatter plot for p-values
    ax1.scatter(regional_df['rel_pos'], regional_df['P'], 
                color='blue', alpha=0.7, s=30)
    
    # Highlight index SNP
    index_row = regional_df[regional_df['SNP'] == snp_id]
    if len(index_row) > 0:
        ax1.scatter(index_row['rel_pos'], index_row['P'], 
                   color='red', marker='*', s=200, 
                   label=f'Index SNP: {snp_id}')
    
    # Significance lines
    ax1.axhline(y=-np.log10(5e-8), color='red', linestyle='--', alpha=0.7)
    ax1.axhline(y=-np.log10(1e-5), color='orange', linestyle='--', alpha=0.7)
    
    # Labels
    ax1.set_xlabel('Position relative to index SNP (bp)', fontsize=12)
    ax1.set_ylabel('-log₁₀(p-value)', fontsize=12)
    ax1.set_title(f'{title}\nChromosome {chr_num}, {snp_id} ± {window_size:,}bp', 
                  fontsize=14, fontweight='bold')
    
    # Format x-axis in kb or Mb
    def format_pos(x, pos):
        if abs(x) >= 1e6:
            return f'{x/1e6:.1f}MB'
        else:
            return f'{x/1000:.0f}KB'
    
    ax1.xaxis.set_major_formatter(plt.FuncFormatter(format_pos))
    ax1.grid(True, alpha=0.3)
    ax1.legend()
    
    # Add recombination rates (simulated)
    ax2 = ax1.twinx()
    
    # Simulate recombination rates
    np.random.seed(42)
    recomb_rates = np.random.uniform(0, 3, len(regional_df))
    ax2.plot(regional_df['rel_pos'], recomb_rates, 'g-', alpha=0.5, linewidth=2)
    ax2.set_ylabel('Recombination rate (cM/Mb)', color='green', fontsize=12)
    ax2.tick_params(axis='y', labelcolor='green')
    
    plt.tight_layout()
    return fig, (ax1, ax2)

# Find most significant SNP
top_snp = gwas_df.loc[gwas_df['P'].idxmax(), 'SNP']

# Create regional plot
regional_fig, axes = create_regional_plot(gwas_df, top_snp, 
                                          window_size=500000)
if regional_fig:
    plt.show()

## Question 3:
What does the regional association plot tell us about the association signal around the most significant SNP? Are there multiple independent signals in this region?

In [ ]:
# Your analysis here
# Examine the regional plot and answer the questions above

## Part 6: Exercise Challenges

### Challenge 1: SNP Annotation
Add SNP annotation to your Manhattan plot. Highlight SNPs that:
- Are in coding regions
- Have predicted functional impact
- Are eQTL variants

### Challenge 2: Power Analysis
Calculate the power of this study to detect associations with different effect sizes and allele frequencies.

### Challenge 3: Meta-Analysis
Combine results from this GWAS with simulated results from additional cohorts and show how meta-analysis improves power.

## Summary

In this exercise, you:
1. ✅ Loaded and explored GWAS summary statistics
2. ✅ Created static and interactive Manhattan plots
3. ✅ Analyzed quality control with QQ plots
4. ✅ Examined regional associations in detail
5. ✅ Interpreted significance thresholds and effect sizes

## Key Takeaways
- Manhattan plots visualize genome-wide association results effectively
- QQ plots help detect systematic bias in GWAS results
- Regional plots provide detailed views of association signals
- Statistical significance doesn't guarantee biological importance
- Multiple testing correction is crucial for GWAS interpretation

## Further Practice
1. Download real GWAS summary statistics from GWAS Catalog
2. Explore different visualization tools (qqman, locuszoom)
3. Try fine-mapping approaches
4. Investigate functional annotations of associated variants